<div class="alert alert-block alert-danger">

# FIT5196 Task 2 in Assessment 1
    
#### Student Name: xxxxxxx
#### Student ID: xxxxxxxxx

Date: xxxxxxxx

Environment: xxxxxx

Libraries used:
* os (for interacting with the operating system, included in Python xxxx) 
* pandas 1.1.0 (for dataframe, installed and imported) 
* multiprocessing (for performing processes on multi cores, included in Python 3.6.9 package) 
* itertools (for performing operations on iterables)
* nltk 3.5 (Natural Language Toolkit, installed and imported)
* nltk.tokenize (for tokenization, installed and imported)
* nltk.stem (for stemming the tokens, installed and imported)

    </div>

<div class="alert alert-block alert-info">
    
## Table of Contents

</div>

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Input File](#examine) <br>
[4. Loading and Parsing Files](#load) <br>
$\;\;\;\;$[4.1. Tokenization](#tokenize) <br>
$\;\;\;\;$[4.2. Whatever else](#whetev) <br>
$\;\;\;\;$[4.3. Genegrate numerical representation](#whetev1) <br>
[5. Writing Output Files](#write) <br>
$\;\;\;\;$[5.1. Vocabulary List](#write-vocab) <br>
$\;\;\;\;$[5.2. Sparse Matrix](#write-sparseMat) <br>
[6. Summary](#summary) <br>
[7. References](#Ref) <br>

<div class="alert alert-block alert-success">
    
## 1.  Introduction  <a class="anchor" name="Intro"></a>

This assessment concerns textual data and the aim is to extract data, process them, and transform them into a proper format. The dataset provided is in the format of a PDF file containing ....

<div class="alert alert-block alert-success">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>

In this assessment, any python packages is permitted to be used. The following packages were used to accomplish the related tasks:

* **os:** to interact with the operating system, e.g. navigate through folders to read files
* **re:** to define and use regular expressions
* **pandas:** to work with dataframes
* **multiprocessing:** to perform processes on multi cores for fast performance 
* ...

In [2]:
import os
import re
import langid
import pandas as pd
import multiprocessing
from itertools import chain
import nltk
from nltk.probability import *
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
from nltk.stem import PorterStemmer
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer

-------------------------------------

<div class="alert alert-block alert-success">
    
## 3.  Examining Input File <a class="anchor" name="examine"></a>

In [17]:
#examine sample input
df = pd.read_json("../Sample Output/task1_sampleoutput.json")
df = df.T.reset_index() 
df.columns = ['gmap_id', 'reviews', 'earliest', 'latest'] 
for index, row in df.iterrows():
    filtered_reviews = [review for review in row["reviews"] if review.get("review_text") != "None"]
    df.at[index, "reviews"] = filtered_reviews

print(df.shape)
def extract_reviews(df):
    df = df[df['reviews'].apply(len) >= 70].reset_index(drop=True)
    print(df)
    df['review_text'] = ''
    for index, row in df.iterrows():
        review_texts = []
        for review in row['reviews']:
            text = review.get('review_text', '').lower()
            # text = re.sub(r'[^\x00-\x7F]+', '', text)
            review_texts.append(text)
        df.at[index, 'review_text'] = ' '.join(review_texts)
    print(df.shape)
    return df

df_review = extract_reviews(df)


(176, 4)
                                  gmap_id  \
0   0x80c32ff0cb5fadc3:0xb80050fad0fcf28e   
1   0x80c2b9af0920e40d:0xaee6e638843e3778   
2   0x80843810081ce52d:0xed9e63802f656fe7   
3   0x80857e82c27f75c9:0x3b3b5cd77d6f371f   
4   0x80857e798a45fe1d:0x3cc6b3be2874dea2   
..                                    ...   
73  0x8090538d0a64d315:0x1bfa592312d80087   
74  0x80dd28a80d3694db:0x1856a806dae34dd7   
75  0x80945f207c23ebd3:0x1260ce760b0a27c3   
76  0x80d9551786119691:0x4f2f0412a8ee712e   
77  0x80dcddb056dafeab:0x86dae8fdfafcc2f2   

                                              reviews             earliest  \
0   [{'user_id': '110112184307670094392', 'time': ...  2018-06-19 18:14:16   
1   [{'user_id': '103441949845857274027', 'time': ...  2017-02-03 06:38:24   
2   [{'user_id': '115950312026745775978', 'time': ...  2016-06-01 15:48:07   
3   [{'user_id': '114389874870768911919', 'time': ...  2012-07-03 00:43:39   
4   [{'user_id': '104816159864697130516', 'time': ...  2012-

In [16]:
df = pd.read_json("task1_sample_30.json")
df = df.T.reset_index() 
df.columns = ['gmap_id', 'reviews', 'earliest', 'latest'] 
print(df)
for index, row in df.iterrows():
    filtered_reviews = [review for review in row["reviews"] if review.get("review_text") != "None"]
    df.at[index, "reviews"] = filtered_reviews

print(df.shape)
def extract_reviews(df):
    df = df[df['reviews'].apply(len) >= 70].reset_index(drop=True)
    df['review_text'] = ''
    for index, row in df.iterrows():
        review_texts = []
        for review in row['reviews']:
            text = review.get('review_text', '').lower()
            # text = re.sub(r'[^\x00-\x7F]+', '', text)
            review_texts.append(text)
        df.at[index, 'review_text'] = ' '.join(review_texts)
    print(df.shape)
    return df

df_review = extract_reviews(df)


                                   gmap_id  \
0    0x5490148aca2c3e9d:0xf5573bee4e65e43c   
1    0x54d067024d499153:0x5957bba66f2cf917   
2    0x54d15a8c9fadd0c5:0x6f3dabf2061ca13f   
3    0x80803548ec1ed4e7:0x82301f8f5a243561   
4    0x80843588fed19c55:0xeaed4339ca3a5967   
..                                     ...   
171  0x80ea421ce86f0497:0x5b5afa077475a774   
172  0x80ea4300f599d8e5:0x7329ab3d671425c1   
173  0x80ea6a91eafe7acd:0x6d9ba9785af38ee0   
174  0x80eb2d286a214de3:0xbf9b24250923ebdc   
175  0x80ecc56bd86d4b75:0xa9e2a577ebba1dde   

                                               reviews             earliest  \
0    [{'user_id': '116224981347936201404', 'time': ...  2015-02-18 04:11:32   
1    [{'user_id': '101841141703065914959', 'time': ...  2019-09-10 04:30:03   
2    [{'user_id': '117474486532753329289', 'time': ...  2011-02-04 10:15:09   
3    [{'user_id': '114379335207252591536', 'time': ...  2012-10-19 07:46:12   
4    [{'user_id': '105759461613667378610', 'time': .

Let's examine what is the content of the file. For this purpose, .... 

It is noteiced that file contains ....

Having parsed the pdf file, the following observations can be made: ....



In [4]:
#some code

In [5]:
#some code to download paper files if needed..

<div class="alert alert-block alert-success">
    
## 4.  Loading and Parsing File <a class="anchor" name="load"></a>

In this section, ....

In [6]:
#some code

Let's examine the dictionary generated. For counting the total number of reviews extracted ....

In [7]:
#some code

<div class="alert alert-block alert-warning">
    
### 4.1. Tokenization <a class="anchor" name="tokenize"></a>

Tokenization is a principal step in text processing and producing unigrams. In this section, ....

In [12]:
#Test version
stopwords_list = []
with open(r'Student Data/stopwords_en.txt', 'r') as file:
    for line in file:
        stopwords_list.append(line.strip())
for index,row in df_review.iterrows():
    # Tokenization
    review_text = row["review_text"]
    tokenizer = RegexpTokenizer(r"[a-zA-Z]+")
    unigram_tokens = tokenizer.tokenize(review_text)

    # Remove tokens less than three
    unigram_tokens = [token for token in unigram_tokens if len(token) >= 3]

    # Remove independent stop words
    stopped_tokens = [w for w in unigram_tokens if w not in stopwords_list]

    #Stemming
    stemmer = PorterStemmer()
    stemed_tokens = [stemmer.stem(token) for token in stopped_tokens]
    df_review.at[index, "tokennized_review"] = stemed_tokens
    # print(mwe_tokens)

# Remove context dependent and rare word
word_in_business = {}
total_businesses = df_review.shape[0]

for index, row in df_review.iterrows():
    gmap_id = row['gmap_id']
    unique_words = set(row['tokennized_review'])
    for word in unique_words:
        if word not in word_in_business:
            word_in_business[word] = set()
        word_in_business[word].add(gmap_id)

rare_context_dependent_stopwords = [word for word, business_set in word_in_business.items()
                                if len(business_set) / total_businesses > 0.95 or len(business_set) / total_businesses < 0.05]

for index,row in df_review.iterrows():
    cleaned_tokens = [w for w in row['tokennized_review'] if w not in rare_context_dependent_stopwords]
    df_review.at[index, "tokennized_review"] = cleaned_tokens

# Bigram words
all_tokens = sum(df_review["tokennized_review"], [])
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_words(all_tokens)
bigram_token = finder.nbest(bigram_measures.pmi, 200)  # Get the top 200 bigrams
tokenizer = MWETokenizer(bigram_token)
for index, row in df_review.iterrows():
    mwe_tokens = tokenizer.tokenize(row["tokennized_review"])
    df_review.at[index, "tokennized_review"] = mwe_tokens  
# print(df_review.head())

# Export word count txt file
all_tokens = sum(df_review["tokennized_review"], [])
all_tokens_per_doc = [token for token in df_review["tokennized_review"]]
print(all_tokens_per_doc)
unique_vocab = set(all_tokens)
sorted_unique_vocab = sorted(unique_vocab)
with open("test_vocab.txt",'w') as file:
    for index,word in enumerate(sorted_unique_vocab):
        file.write(f"{word}:{index}\n")




[['found', 'long', 'search', 'complex', 'maintain', 'manag', 'respons', 'issu', 'special', 'mention', 'made', 'move', 'move', 'smooth', 'ensur', 'troubl', 'thank', 'idea', 'check', 'neighbourhood', 'ensur', 'comfort', 'person', 'haven', 'issu', 'enjoy', 'stay', 'agent', 'help', 'graciou', 'accommod', 'adequ', 'answer', 'question', 'explain', 'wasn', 'clear', 'satisfi', 'stay', 'feel', 'secur', 'stay', 'locat', 'conveni', 'resid', 'month', 'bit', 'experi', 'made', 'move', 'commun', 'smoother', 'easier', 'experienc', 'hospit', 'team', 'son', 'feel', 'home', 'extrem', 'safe', 'maintain', 'team', 'leas', 'agent', 'secur', 'special', 'move', 'apart', 'expect', 'guid', 'tour', 'month', 'ago', 'decid', 'apart', 'apart', 'leas', 'agent', 'help', 'applic', 'process', 'deal', 'downsid', 'apt', 'fan', 'open', 'tini', 'window', 'air', 'flow', 'open', 'shower', 'door', 'water', 'floor', 'flow', 'carpet', 'apart', 'encourag', 'renter', 'home', 'comfort', 'appli', 'sort', 'decor', 'wall', 'wall', 'fu

In [11]:
#Correct version
stopwords_list = []
with open(r'Student Data/stopwords_en.txt', 'r') as file:
    for line in file:
        stopwords_list.append(line.strip())
for index,row in df_review.iterrows():
    # Tokenization
    review_text = row["review_text"]
    tokenizer = RegexpTokenizer(r"[a-zA-Z]+")
    unigram_tokens = tokenizer.tokenize(review_text)

    # Remove tokens less than three
    unigram_tokens = [token for token in unigram_tokens if len(token) >= 3]

    # Remove independent stop words
    stopped_tokens = [w for w in unigram_tokens if w not in stopwords_list]

    #Stemming
    stemmer = PorterStemmer()
    stemed_tokens = [stemmer.stem(token) for token in stopped_tokens]
    df_review.at[index, "tokennized_review"] = stemed_tokens
    # print(mwe_tokens)

# Remove context dependent and rare word
word_in_business = {}
total_businesses = df_review.shape[0]

for index, row in df_review.iterrows():
    gmap_id = row['gmap_id']
    unique_words = set(row['tokennized_review'])
    for word in unique_words:
        if word not in word_in_business:
            word_in_business[word] = set()
        word_in_business[word].add(gmap_id)

rare_context_dependent_stopwords = [word for word, business_set in word_in_business.items()
                                if len(business_set) / total_businesses > 0.95 or len(business_set) / total_businesses < 0.05]

for index,row in df_review.iterrows():
    cleaned_tokens = [w for w in row['tokennized_review'] if w not in rare_context_dependent_stopwords]
    df_review.at[index, "tokennized_review"] = cleaned_tokens

# Bigram words
all_tokens = sum(df_review["tokennized_review"], [])
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_words(all_tokens)
bigram_token = finder.nbest(bigram_measures.pmi, 200)  # Get the top 200 bigrams
tokenizer = MWETokenizer(bigram_token)
for index, row in df_review.iterrows():
    mwe_tokens = tokenizer.tokenize(row["tokennized_review"])
    df_review.at[index, "tokennized_review"] = mwe_tokens  
# print(df_review.head())

# Export word count txt file
all_tokens = sum(df_review["tokennized_review"], [])
all_tokens_per_doc = [token for token in df_review["tokennized_review"]]
print(all_tokens_per_doc)
unique_vocab = set(all_tokens)
sorted_unique_vocab = sorted(unique_vocab)
with open("test_vocab.txt",'w') as file:
    for index,word in enumerate(sorted_unique_vocab):
        file.write(f"{word}:{index}\n")

[['found', 'long', 'search', 'complex', 'maintain', 'manag', 'respons', 'issu', 'special', 'mention', 'made', 'move', 'move', 'smooth', 'ensur', 'troubl', 'thank', 'idea', 'check', 'neighbourhood', 'ensur', 'comfort', 'person', 'haven', 'issu', 'enjoy', 'stay', 'agent', 'help', 'graciou', 'accommod', 'adequ', 'answer', 'question', 'explain', 'wasn', 'clear', 'satisfi', 'stay', 'feel', 'secur', 'stay', 'locat', 'conveni', 'resid', 'month', 'bit', 'experi', 'made', 'move', 'commun', 'smoother', 'easier', 'experienc', 'hospit', 'team', 'son', 'feel', 'home', 'extrem', 'safe', 'maintain', 'team', 'leas', 'agent', 'secur', 'special', 'move', 'apart', 'expect', 'guid', 'tour', 'month', 'ago', 'decid', 'apart', 'apart', 'leas', 'agent', 'help', 'applic', 'process', 'deal', 'downsid', 'apt', 'fan', 'open', 'tini', 'window', 'air', 'flow', 'open', 'shower', 'door', 'water', 'floor', 'flow', 'carpet', 'apart', 'encourag', 'renter', 'home', 'comfort', 'appli', 'sort', 'decor', 'wall', 'wall', 'fu

In [ ]:
#some code
for index,row in df_review.iterrows():
    # Tokenization
    review_text = row["review_text"]
    tokenizer = RegexpTokenizer(r"[a-zA-Z]+")
    unigram_tokens = tokenizer.tokenize(review_text)

    # Remove tokens less than three
    unigram_tokens = [token for token in unigram_tokens if len(token) >= 3]

    # Retokenize for multi word
    bigrams = ngrams(unigram_tokens, n = 2)
    fdbigram = FreqDist(bigrams)
    fdbigram.most_common()
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    finder = nltk.collocations.BigramCollocationFinder.from_words(unigram_tokens)
    bigram_token = finder.nbest(bigram_measures.pmi, 201)
    tokenizer = MWETokenizer(bigram_token)
    mwe_tokens = tokenizer.tokenize(unigram_tokens)
    # print(mwe_tokens)

    # Remove independent stop words
    stopwords_list = []
    with open(r'Student Data/stopwords_en.txt', 'r') as file:
        for line in file:
            stopwords_list.append(line.strip())
    stopped_tokens = [w for w in mwe_tokens if w not in stopwords_list]

    # Stem words
    stemmer = PorterStemmer()
    stemed_tokens = [stemmer.stem(token) for token in stopped_tokens]

    df_review.at[index, "tokennized_review"] = stemed_tokens


# print(df_review.head())

# Remove context dependent and rare word
word_in_business = {}
total_businesses = df_review.shape[0]

for index, row in df_review.iterrows():
    gmap_id = row['gmap_id']
    unique_words = set(row['tokennized_review'])
    for word in unique_words:
        if word not in word_in_business:
            word_in_business[word] = set()
        word_in_business[word].add(gmap_id)

rare_context_dependent_stopwords = [word for word, business_set in word_in_business.items()
                                if len(business_set) / total_businesses > 0.95 or len(business_set) / total_businesses < 0.05]

for index,row in df_review.iterrows():
    cleaned_tokens = [w for w in row['tokennized_review'] if w not in rare_context_dependent_stopwords]
    df_review.at[index, "tokennized_review"] = cleaned_tokens

# print(df_review.head())

# Export word count txt file
all_tokens = sum(df_review["tokennized_review"], [])
unique_vocab = set(all_tokens)
sorted_unique_vocab = sorted(unique_vocab)
with open("test_vocab.txt",'w') as file:
    for index,word in enumerate(sorted_unique_vocab):
        file.write(f"{word}:{index}\n")










ValueError: Must have equal len keys and value when setting with an iterable

The above operation results in a dictionary with PID representing keys and a single string for all reviews of the day concatenated to each other. ...

In [ ]:
#Remove

At this stage, all reviews for each PID are tokenized and are stored as a value in the new dictionary (separetely for each day).

-------------------------------------

<div class="alert alert-block alert-warning">
    
### 4.2. Whatever else <a class="anchor" name="whetev"></a>

<div class="alert alert-block alert-warning">
    
### 4.3. Generate numerical representation<a class="anchor" name="bigrams"></a>

One of the tasks is to generate the numerical representation for all tokens in abstract.  .....

In [ ]:
#some code
vectorizer = CountVectorizer(analyzer="word")
documents = [' '.join(tokens) for tokens in all_tokens_per_doc]

data_features = vectorizer.fit_transform(documents)

vocab = vectorizer.get_feature_names_out()

vocab_dist = {word: index for index, word in enumerate(vocab)}
output_countvec = []
for doc_index, gmap_id in enumerate(df_review["gmap_id"]):
    doc_vector = data_features[doc_index]
    token_counts = zip(doc_vector.indices, doc_vector.data)
    formatted_counts = [f"{index}:{frequency}" for index,frequency in token_counts] 
    output_line = f"{gmap_id} " + ", ".join(formatted_counts)
    output_countvec.append(output_line)

with open("countvec_test.txt","w") as file:
    for line in output_countvec:
        file.write(line + "\n")

In [ ]:
#some code

Random descriptions and justification ...

In [ ]:
#some code

At this stage, we have a dictionary of tokenized words, whose keys are indicative of....

-------------------------------------

#### Whatever else <a class="anchor" name="whatev1"></a>

<div class="alert alert-block alert-success">
    
## 5. Writing Output Files <a class="anchor" name="write"></a>

files need to be generated:
* Vocabulary list
* Sparse matrix (count_vectors)

This is performed in the following sections.

<div class="alert alert-block alert-warning">
    
### 5.1. Vocabulary List <a class="anchor" name="write-vocab"></a>

List of vocabulary should also be written to a file, sorted alphabetically, with their reference codes in front of them. This file also refers to the sparse matrix in the next file. For this purpose, .....

In [ ]:
#some code

<div class="alert alert-block alert-warning">
    
### 5.2. Sparse Matrix <a class="anchor" name="write-sparseMat"></a>

For writing sparse matrix for a paper, we firstly calculate the frequency of words for that paper ....

In [ ]:
#some code

-------------------------------------

<div class="alert alert-block alert-success">
    
## 6. Summary <a class="anchor" name="summary"></a>

.....

-------------------------------------

<div class="alert alert-block alert-success">
    
## 7. References <a class="anchor" name="Ref"></a>

[1] Pandas dataframe.drop_duplicates(), https://www.geeksforgeeks.org/python-pandas-dataframe-drop_duplicates/, Accessed 13/08/2022.



## --------------------------------------------------------------------------------------------------------------------------